# CHB MIT DATABASE

1. Load edf data
2. define class sz/nsz
3. insert class cols
4. concat
5. windowing
6. ensemple learn resnet152, inceptionv3, incep+resnet
7. define OHEM loss
8. build network
9. evaluate
10. calculate for confussion matrices (f1-score etc.)

In [4]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
%matplotlib inline

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


In [4]:
cwd = os.getcwd()
cwd

'/Users/niklashjort/Desktop/Notes/Speciale/projects'

In [3]:
database_path = 'Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/'
patient_one_path = 'chb01/'

def GetAllEdfFileNames(directory):
    fileType = ".edf"
    files = os.listdir(directory)
    edfFileNameList = [(directory + "/" + x) for x in files if (x.endswith(fileType))]
    return edfFileNameList

EEG_dataset= GetAllEdfFileNames(database_path + patient_one_path)

for file in EEG_dataset[10:15]:
    print(file)

print(type(EEG_dataset[1]))

Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/chb01//chb01_22.edf
Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/chb01//chb01_36.edf
Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/chb01//chb01_37.edf
Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/chb01//chb01_23.edf
Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/chb01//chb01_27.edf
<class 'str'>


In [5]:
def ReadEdfFile(FileName, print_reader_info = False):
    print("current_file: " + FileName)
    try:
        if(print_reader_info):
            data = mne.io.read_raw_edf(FileName, verbose=True)
        else:
            data = mne.io.read_raw_edf(FileName, verbose=False)

        raw_data = data.get_data()
        converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
        return converted_raw
    except:
        print("error: something went wrong with edf-reading")

ch01_df = ReadEdfFile(EEG_dataset[11])
ch01_df.head()

current_file: Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/chb01//chb01_36.edf


,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,F4-C4,...,F8-T8,T8-P8,P8-O2,FZ-CZ,CZ-PZ,P7-T7,T7-FT9,FT9-FT10,FT10-T8,STI 014
0,-3.496947e-05,8.068376e-05,-1.777778e-05,1.387057e-05,-1.170208e-04,-1.230769e-05,5.177045e-05,1.193651e-04,-1.650794e-04,1.275702e-04,...,1.041270e-04,5.489621e-05,-1.853968e-04,-1.545299e-04,4.708181e-05,1.816850e-05,-7.130647e-05,-1.699634e-05,9.006105e-05,54.0
1,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0
2,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,-1.953602e-07,1.953602e-07,5.860806e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,-1.953602e-07,-5.860806e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0
3,1.953602e-07,1.953602e-07,-5.860806e-07,1.953602e-07,1.953602e-07,-3.711844e-06,1.953602e-07,3.321123e-06,-5.860806e-07,1.953602e-07,...,1.758242e-06,-9.768010e-07,-2.539683e-06,-4.493284e-06,9.768010e-07,9.768010e-07,-1.367521e-06,5.860806e-07,1.758242e-06,0.0
4,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,-1.953602e-07,-5.860806e-07,1.758242e-06,1.953602e-07,-1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,5.860806e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0


## Memory Usage & Loading Time

In [6]:
print(ch01_df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921600 entries, 0 to 921599
Data columns (total 23 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   FP1-F7    921600 non-null  float64
 1   F7-T7     921600 non-null  float64
 2   T7-P7     921600 non-null  float64
 3   P7-O1     921600 non-null  float64
 4   FP1-F3    921600 non-null  float64
 5   F3-C3     921600 non-null  float64
 6   C3-P3     921600 non-null  float64
 7   P3-O1     921600 non-null  float64
 8   FP2-F4    921600 non-null  float64
 9   F4-C4     921600 non-null  float64
 10  C4-P4     921600 non-null  float64
 11  P4-O2     921600 non-null  float64
 12  FP2-F8    921600 non-null  float64
 13  F8-T8     921600 non-null  float64
 14  T8-P8     921600 non-null  float64
 15  P8-O2     921600 non-null  float64
 16  FZ-CZ     921600 non-null  float64
 17  CZ-PZ     921600 non-null  float64
 18  P7-T7     921600 non-null  float64
 19  T7-FT9    921600 non-null  float64
 20  FT9-

In [8]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

print(mem_usage(ch01_df))

161.72 MB


## Add col ['timems', class] - Reduce dtypes - Compress + Save

Adding cols

In [9]:
pd.io.parquet.get_engine('auto').__class__

pandas.io.parquet.FastParquetImpl

Reduce dtypes, save memory, uptimize loading times

In [7]:
def downcast_dtypes(df):
    _start = df.memory_usage(deep=True).sum() / 1024 ** 2
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    _end = df.memory_usage(deep=True).sum() / 1024 ** 2
    saved_time = (_start - _end) / _start * 100
    print(f"Saved: {saved_time:.2f}%")
    return df

Insert time ms in dataframe function

In [106]:
import re

summary_file_path = database_path + patient_one_path + 'chb01-summary.txt'


timestamp_info_pattern = "Number of Seizures$"

str1 = ""

with open(summary_file_path, 'r') as f:
    for line in f:
        str1 += str(line).replace("\n", "<br>")

Get sz information for [patient]-summary.txt files. </br>
Then get information on seizures timestamp and the file name </br>

In [165]:
s = re.findall('<br><br>(.*?)<br><br>', str1)

for index, line in enumerate(s):
    if re.match('(^Channels in EDF Files:|^Channels changed:)', line):
        s.remove(s[index])
    else:
        pass

File Name: chb01_18.edf File Start Time: 04:45:06 File End Time: 5:45:06 Number of Seizures in File: 1 Seizure Start Time: 1720 seconds Seizure End Time: 1810 seconds
------------props----------------
chb01_18
04:45:06


In [168]:
class FileInformationContainer:
    def __init__(self, information_str):
        self.information_str = self.clean_string(information_str)
        self.file_name = self.set_filename()
        self.time_start = self.set_file_time_start_ms()
        self.sz_count = None
        self.sz_info = []
        
    def clean_string(self, uncleaned_str):
        return uncleaned_str.replace("<br>", " ")

    def set_filename(self):
        filename_found = re.match(r"^File Name: (.+?).edf", self.information_str)
        if filename_found:
            return filename_found.group(1)
        else:
            print(f"{self.file_name} failed get_filename")
            return "filename not found"
    
    def get_milli_sec(self, time_str):
        """Get Seconds from time."""
        h, m, s = time_str.split(':')
        return (int(h) * 3600 + int(m) * 60 + int(s)) * 1000

    def set_file_time_start_ms(self):
        time_start_found = re.match(r".*File Start Time: (.*?) File", self.information_str)
        if time_start_found:
            try:
                return self.get_milli_sec(time_start_found.group(1))
            except Exception as e :
                print(f"{self.file_name}: error {e} cannot convert to ms time")
                return f"{e}"
        else:
            print(f"{self.file_name} failed get_file_time_start_ms")
            return "time start not found"
    
    

    def get(self):
        return self.information_str


obj = FileInformationContainer(s[8])
print(obj.get())
print("------------props----------------")
print(obj.file_name)
print(obj.time_start)

File Name: chb01_18.edf File Start Time: 04:45:06 File End Time: 5:45:06 Number of Seizures in File: 1 Seizure Start Time: 1720 seconds Seizure End Time: 1810 seconds
------------props----------------
chb01_18
17106000


Compress & save

In [8]:
filename = "hej2"
def df_save_compress(path, patient, filename, df):
    df.to_parquet(f"{path}{patient}{filename}.parquet.gzip", compression="gzip")

df_save_compress(database_path, patient_one_path, filename, ch01_df)

# Read & load compressed dataframes

In [12]:
compressed_file_type = ".parquet.gzip"
compressed_df = pd.read_parquet(database_path + patient_one_path + filename + compressed_file_type)

In [13]:
type(compressed_df)

pandas.core.frame.DataFrame